## Guia MLflow

In [12]:
import mlflow
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [27]:
import mlflow
from mlflow.models import infer_signature
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

# Load the Iris dataset
X, y = datasets.load_iris(return_X_y=True)
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
# Define the model hyperparameters
params = {
    "solver": "lbfgs",
    "max_iter": 1000,
    "multi_class": "auto",
    "random_state": 8888,
}
# Train the model
lr = LogisticRegression(**params)
lr.fit(X_train, y_train)
# Predict on the test set
y_pred = lr.predict(X_test)
# Calculate metrics
accuracy = accuracy_score(y_test, y_pred)

# se crean nuevas métricas
precision = precision_score(y_test, y_pred, average="macro")
recall = recall_score(y_test, y_pred, average="macro")
f1 = f1_score(y_test, y_pred, average="macro")
y_proba = lr.predict_proba(X_test)
AUC_ROC = roc_auc_score(y_test, y_proba, multi_class="ovr")


/Users/henryragua/Documents/estudio/MachineLearning/Semestre III/MlOps/mlflow/pythonProject/.venv/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


In [28]:
# Set our tracking server uri for logging
mlflow.set_tracking_uri(uri="http://127.0.0.1:8080")

In [30]:


# Create a new MLflow Experiment
mlflow.set_experiment("MLflow Quickstart")
# Start an MLflow run
with mlflow.start_run():
    # Log the hyperparameters
    mlflow.log_params(params)
    
    # Log the loss metric
    mlflow.set_tag("Training Info", "Basic LR model for iris data")
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("precision", precision)
    mlflow.log_metric("recall", recall)
    mlflow.log_metric("f1", f1)
    mlflow.log_metric("AUC_ROC", AUC_ROC)
    # Infer the model signature
    signature = infer_signature(X_train, lr.predict(X_train))
    # Log the model
    model_info = mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2025/04/05 14:58:43 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 5


🏃 View run mercurial-ant-607 at: http://127.0.0.1:8080/#/experiments/154373928185873316/runs/fc8cbadbc11648ccb9421c761366269f
🧪 View experiment at: http://127.0.0.1:8080/#/experiments/154373928185873316


Created version '5' of model 'tracking-quickstart'.


In [31]:
# Load the model back for predictions as a generic Python Function model
loaded_model = mlflow.pyfunc.load_model(model_info.model_uri)

predictions = loaded_model.predict(X_test)
iris_feature_names = datasets.load_iris().feature_names
result = pd.DataFrame(X_test, columns=iris_feature_names)
result["actual_class"] = y_test
result["predicted_class"] = predictions
result[:4]

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),actual_class,predicted_class
0,6.1,2.8,4.7,1.2,1,1
1,5.7,3.8,1.7,0.3,0,0
2,7.7,2.6,6.9,2.3,2,2
3,6.0,2.9,4.5,1.5,1,1


In [32]:
import mlflow.pyfunc
import numpy as np
# Ruta del modelo registrado
model_uri = "mlflow-artifacts:/455314737741369698/a40bf7dbf9da4e5ba12a4ec42b77de53/artifacts/iris_model"

loaded_model = mlflow.pyfunc.load_model(model_uri)
# Define new test data
new_data = np.array([
    [5.1, 3.5, 1.4, 0.2],
    [6.7, 3.1, 4.7, 1.5]
])
# Use the loaded model to make predictions
predictions = loaded_model.predict(new_data)
# Print the prediction results
print("Predictions:", predictions)

MlflowException: The following failures occurred while downloading one or more artifacts from http://127.0.0.1:8080/api/2.0/mlflow-artifacts/artifacts/455314737741369698/a40bf7dbf9da4e5ba12a4ec42b77de53/artifacts:
##### File iris_model #####
API request to http://127.0.0.1:8080/api/2.0/mlflow-artifacts/artifacts/455314737741369698/a40bf7dbf9da4e5ba12a4ec42b77de53/artifacts/iris_model failed with exception HTTPConnectionPool(host='127.0.0.1', port=8080): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/455314737741369698/a40bf7dbf9da4e5ba12a4ec42b77de53/artifacts/iris_model (Caused by ResponseError('too many 500 error responses'))

In [ ]:
import pandas as pd
import mlflow.pyfunc
import argparse


# Cargar datos
data = pd.read_csv('/home/manuelcastiblan/academic/iris_data.csv')

# Cargar modelo
model = mlflow.pyfunc.load_model('mlflow-artifacts:/455314737741369698/a40bf7dbf9da4e5ba12a4ec42b77de53/artifacts/iris_model')

# Hacer predicciones
predictions = model.predict(data)

# Mostrar resultados
print("\n🧪 Resultados de predicción:")
for i, pred in enumerate(predictions):
    print(f"Muestra {i+1}: Clase predicha → {pred}")



🧪 Resultados de predicción:
Muestra 1: Clase predicha → 0
Muestra 2: Clase predicha → 1
Muestra 3: Clase predicha → 2
